# About the Game

The rules of 3 Snails Isolation are a variation of the original Isolation. In the original form of the game there are two players, each with their own game piece, and a 7-by-7 grid of squares. At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, the players alternate turns moving their piece like a queen in chess (any number of open squares vertically, horizontally, or diagonally). When the piece is moved, the square that was previously occupied is blocked, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this 3 Snails variant, each player controls 3 pieces ("snails"), each of which can only move to 1 surrounding square, either to the left/right or directly above/below it. Once a piece moves away from a square, no other piece can occupy the square again, as in the original Isolation game. For clarity, examine the scenario below:

In the image below, both Q1 and Q2 place their snails on the board. Note, that when choosing your move, you must ensure your moves are selected in proper order


![](./img/sn_1.png)

In the image below, Q1 is in the process of choosing its next set of moves. It has chosen 2 moves already, which are printed in the output box, and still needs to pick where "13" will move. Note that the possible moves for any given piece is either to the left/down or directly above/below it.

![](./img/sn_2.png)

Q1 makes its move and Q2 has already chosen the next set of moves. In the image below, Q2 tried to move two of its snails to the same location. Such moves are illegal, as indicated by the message in the output box.

![](./img/sn_3.png)



You can try playing the game against the Random Player or yourself using the interactive tool below.

# Environment Setup

In [ ]:
%run helpers/verify_config.py # verify the environment setup

In [38]:
# Following two lines make sure anything imported from .py scripts 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)
%reload_ext autoreload

# %load_ext autoreload
# %autoreload 2

# module_path = os.path.abspath(os.path.join('..'))
# module_path
# if module_path not in sys.path:
#     sys.path.append(module_path)

import sys, os
# sys.path.append(os.path.join(os.path.abspath(os.getcwd()), "."))
import os
import sys

# directory_path = os.path.abspath(os.path.join('..'))
# if directory_path not in sys.path:
#     sys.path.append(directory_path)

from board_viz import ReplayGame, InteractiveGame
from interactive_board_viz import PlayInteractiveGame
from isolation import Board
from test_players import RandomPlayer, HumanPlayer, Player
from custom_player import CustomPlayer
from evaluation_functions import (
    OpenMoveEvalFn,
    DefensiveEvalFn,
    OffensiveEvalFn,
    DefenseToOffenseEvalFn,
    OffenseToDefenseEvalFn,
)
# Heuristic for evluation of the board state
from custom_player import CustomPlayer


import time

from isolation import Board
import player_submission_tests as tests

import time
import platform
# import io
from io import StringIO


import copy
from time import sleep
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Label, Button, GridspecLayout
from ipywidgets import Button, GridBox, Layout, ButtonStyle
from IPython.display import display, clear_output

In [39]:

# import resource
if platform.system() != 'Windows':
    import resource

class OpenMoveEvalFn:
    def score(self, game, my_player=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """
        # Simple crude heuristic which favours the number of possible moves our AI has in comparison
        # to the other player. CustomPlayer is always trying to maximize this value
        # and RandomPlayer is trying to minimimize
        if my_player:

            num_active_moves_my_player = game.get_player_moves(my_player=my_player)
            num_active_moves_opponent = game.get_opponent_moves(my_player=my_player)

        else:
            num_active_moves_my_player = game.get_opponent_moves(my_player=my_player)
            num_active_moves_opponent = game.get_player_moves(my_player=my_player)

        return len(num_active_moves_my_player) - len(num_active_moves_opponent)


class DefensiveEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.

        The defensive heuristic favors maximizing the player’s available moves at a weighted
        cost against those available to the opponent. This has the effect of strongly
        preferring to keep the player’s moves larger at all costs against the opponent

        Eval Function == Number of my_player moves * 2 - my_opponent moves

        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.

        Returns:
            float: The current state's score, based on your own heuristic.
        """

        if isinstance(my_player, CustomPlayer):
            my_moves = game.get_player_moves(my_player=my_player)
            opp_moves = game.get_opponent_moves(my_player=my_player)

        else:
            my_moves = game.get_opponent_moves(my_player=my_player)
            opp_moves = game.get_player_moves(my_player=my_player)

        return (len(my_moves) * 2) - len(opp_moves)


class OffensiveEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.

        By contrast to the defensive strategy, the offensive heuristic favors minimizing the
        opponent’s available moves at a weighted cost against those available to the player.
        This achieves an aggressive style of game play, attempting to force or limit the
        opponent into a weaker game position

        Eval Function == Number of my_player moves - my_opponent moves * 2

        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.

        Returns:
            float: The current state's score, based on your own heuristic.
        """

        if isinstance(my_player, CustomPlayer):
            my_moves = game.get_player_moves(my_player=my_player)
            opp_moves = game.get_opponent_moves(my_player=my_player)

        else:
            my_moves = game.get_opponent_moves(my_player=my_player)
            opp_moves = game.get_player_moves(my_player=my_player)

        return len(my_moves) - (len(opp_moves) * 2)


class DefenseToOffenseEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.

        The defensive to offensive heuristic applies an early-game defensive strategy, where the
        player tries to initially maximize the number of available moves. After a period half-way
        through the game, the heuristic switches to offensive, attempting to limit and block
        the opponent.

        This is achieved by taking into account a ratio value, computed from the current round
        divided by the game board size. When the number of rounds in the game is less than
        half-way through, the heuristic employs a defensive strategy. Afterwards, an
        offensive strategy is utilized

        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.

        Returns:
            float: The current state's score, based on your own heuristic.
        """

        if isinstance(my_player, CustomPlayer):
            my_moves = game.get_player_moves(my_player=my_player)
            opp_moves = game.get_opponent_moves(my_player=my_player)

        else:
            my_moves = game.get_opponent_moves(my_player=my_player)
            opp_moves = game.get_player_moves(my_player=my_player)

        board_size = game.width * game.height
        ratio = game.move_count / board_size

        if ratio <= 0.5:
            return (len(my_moves) * 2) - len(opp_moves)
        else:
            return len(my_moves) - (len(opp_moves) * 2)


class OffenseToDefenseEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.

        The offensive to defensive heuristic applies an early-game offensive strategy, where the
        player tries to initially minimize the number of available moves for the opponent. After
        a period half-way through the game, the heuristic switches to defensive, attempting to
        maximize the number of available moves open to the AI agent

        This is achieved by taking into account a ratio value, computed from the current round
        divided by the game board size. When the number of rounds in the game is less than
        half-way through, the heuristic employs a defensive strategy. Afterwards, an
        offensive strategy is utilized

        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.

        Returns:
            float: The current state's score, based on your own heuristic.
        """

        if isinstance(my_player, CustomPlayer):
            my_moves = game.get_player_moves(my_player=my_player)
            opp_moves = game.get_opponent_moves(my_player=my_player)

        else:
            my_moves = game.get_opponent_moves(my_player=my_player)
            opp_moves = game.get_player_moves(my_player=my_player)

        board_size = game.width * game.height
        ratio = game.move_count / board_size

        if ratio <= 0.5:
            return len(my_moves) - (len(opp_moves) * 2)
        else:
            return (len(my_moves) * 2) - len(opp_moves)


# Interactive Game & Playback

In [41]:
def get_details(name):
    if name in {'11','12','13'}:
        color = 'SpringGreen'
    elif name in {'21','22','23'}:
        color = 'tomato'
    elif name == 'q1':
        color = '#bdffbd'
        name = ' '
    elif name == 'q2':
        color = '#ffb6ae'
        name = ' '
    elif name == 'X':
        color = 'black'
    elif name == 'O':
        color = 'orange'
        name = ' '
    else:
        color = 'Lavender'
    style = ButtonStyle(button_color=color)
    return name, style

def create_cell(button_name='', grid_loc=None, click_callback=None):
    layout = Layout(width='auto', height='auto')
    name, style = get_details(button_name)
    button = Button(description=name,layout = layout, style=style)
    button.x, button.y = grid_loc
    if click_callback: button.on_click(click_callback)
    return button

def get_viz_board_state(game, show_legal_moves):
    board_state = game.get_state()
    legal_moves = game.get_active_moves()
    active_player = 'q1' if game.__active_player__ is game.__player_1__ else 'q2'
    if show_legal_moves:
        for moves in legal_moves:
            for r,c in moves:
                board_state[r][c] = active_player
    return board_state

def create_board_gridbox(game, show_legal_moves, click_callback=None):
    h, w = game.height, game.width
    board_state = get_viz_board_state(game, show_legal_moves)

    grid_layout = GridspecLayout(n_rows=h,
                                 n_columns=w,
                                 grid_gap='2px 2px',
                                 width='480px',
                                 height='480px',
                                 justify_content='center'
                                )
    for r in range(h):
        for c in range(w):
            cell = create_cell(board_state[r][c], grid_loc=(r,c), click_callback=click_callback)
            grid_layout[r,c] = cell

    return grid_layout

class InteractiveGame():
    """This class is used to play the game interactively (only works in jupyter)"""
    def __init__(self, player1 = Player("Player1"), opponent=Player("Player2"), show_legal_moves=False):
        self.game = Board(player1, opponent)
        self.width = self.game.width
        self.height = self.game.height
        self.show_legal_moves = show_legal_moves
        self.__click_count = 0
        self.__move = []
        self.gridb = create_board_gridbox(self.game,
                                          self.show_legal_moves,
                                          click_callback=self.select_move)
        self.visualized_state = None
        self.opponent = opponent
        self.output_section = widgets.Output(layout={'border': '1px solid black'})
        self.game_is_over = False
        display(self.gridb)
        display(self.output_section)

    def __reset_turn(self):
        self.__click_count = 0
        self.__move = []
        self.output_section.clear_output()

    def select_move(self, b):
        if platform.system() == 'Windows':
            def curr_time_millis():
                return int(round(time.time() * 1000))
        else:
            def curr_time_millis():
                return 1000 * resource.getrusage(resource.RUSAGE_SELF).ru_utime
        move_start = curr_time_millis()

        def time_left(time_limit = 1000):
            # print("Limit: "+str(time_limit) +" - "+str(curr_time_millis()-move_start))
            return time_limit - (curr_time_millis() - move_start)

        self.__move.append((b.x, b.y))
        with self.output_section:
            print(f"Move {self.__click_count+1}: {(b.x, b.y)}")
        if self.__click_count < 2:
            self.__click_count += 1
            return

        if self.game_is_over:
            with self.output_section:
                print('The game is over!')
            return
        ### swap move workaround ###
        # find if current location is in the legal moves
        # legal_moves is of length 1 if move exists, and len 0 if move is illegal
        moves = self.game.get_active_moves()
        legal_moves = [(x,y,z) for x,y,z in moves if [x,y,z] == self.__move]
        if not legal_moves:
            output = f"move {self.__move} is illegal!"
            self.__reset_turn()
            with self.output_section:
                print(output)
            return
        else:
            # there is only one move in swap isolation game
            self.__move = legal_moves[0]
        ### swap move workaround end ###
        self.game_is_over, winner = self.game.__apply_move__(self.__move)
        if (not self.game_is_over) and (type(self.opponent) != Player):
            opponents_legal_moves = self.game.get_active_moves()
            opponent_move = self.opponent.move(self.game, time_left=time_left)
            assert opponent_move in opponents_legal_moves, \
            f"Opponents move {opponent_move} is not in list of legal moves {opponents_legal_moves}"
            self.game_is_over, winner = self.game.__apply_move__(opponent_move)
        if self.game_is_over: print(f"Game is over, the winner is: {winner}")
        board_vis_state = get_viz_board_state(self.game, self.show_legal_moves)
        for r in range(self.height):
            for c in range(self.width):
                new_name, new_style = get_details(board_vis_state[r][c])
                self.gridb[r,c].description = new_name
                self.gridb[r,c].style = new_style
        self.__reset_turn()
        
ig = InteractiveGame(RandomPlayer(), HumanPlayer())

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

Output(layout=Layout(border='1px solid black'))

[1] ((0, 1), (0, 2), (0, 0))
[2] ((0, 1), (0, 3), (0, 0))
[3] ((0, 1), (0, 4), (0, 0))
[4] ((0, 1), (0, 5), (0, 0))
[5] ((0, 1), (0, 6), (0, 0))
[6] ((0, 1), (1, 0), (0, 0))
[7] ((0, 1), (1, 2), (0, 0))
[8] ((0, 1), (1, 3), (0, 0))
[9] ((0, 1), (1, 4), (0, 0))
[10] ((0, 1), (1, 5), (0, 0))
[11] ((0, 1), (1, 6), (0, 0))
[12] ((0, 1), (2, 0), (0, 0))
[13] ((0, 1), (2, 1), (0, 0))
[14] ((0, 1), (2, 2), (0, 0))
[15] ((0, 1), (2, 3), (0, 0))
[16] ((0, 1), (2, 4), (0, 0))
[17] ((0, 1), (2, 5), (0, 0))
[18] ((0, 1), (2, 6), (0, 0))
[19] ((0, 1), (3, 0), (0, 0))
[20] ((0, 1), (3, 2), (0, 0))
[21] ((0, 1), (3, 3), (0, 0))
[22] ((0, 1), (3, 5), (0, 0))
[23] ((0, 1), (3, 6), (0, 0))
[24] ((0, 1), (4, 0), (0, 0))
[25] ((0, 1), (4, 1), (0, 0))
[26] ((0, 1), (4, 2), (0, 0))
[27] ((0, 1), (4, 3), (0, 0))
[28] ((0, 1), (4, 4), (0, 0))
[29] ((0, 1), (4, 5), (0, 0))
[30] ((0, 1), (4, 6), (0, 0))
[31] ((0, 1), (5, 0), (0, 0))
[32] ((0, 1), (5, 1), (0, 0))
[33] ((0, 1), (5, 2), (0, 0))
[34] ((0, 1), (5, 3

[2215] ((0, 6), (2, 3), (0, 1))
[2216] ((0, 6), (2, 4), (0, 1))
[2217] ((0, 6), (2, 5), (0, 1))
[2218] ((0, 6), (2, 6), (0, 1))
[2219] ((0, 6), (3, 0), (0, 1))
[2220] ((0, 6), (3, 2), (0, 1))
[2221] ((0, 6), (3, 3), (0, 1))
[2222] ((0, 6), (3, 5), (0, 1))
[2223] ((0, 6), (3, 6), (0, 1))
[2224] ((0, 6), (4, 0), (0, 1))
[2225] ((0, 6), (4, 1), (0, 1))
[2226] ((0, 6), (4, 2), (0, 1))
[2227] ((0, 6), (4, 3), (0, 1))
[2228] ((0, 6), (4, 4), (0, 1))
[2229] ((0, 6), (4, 5), (0, 1))
[2230] ((0, 6), (4, 6), (0, 1))
[2231] ((0, 6), (5, 0), (0, 1))
[2232] ((0, 6), (5, 1), (0, 1))
[2233] ((0, 6), (5, 2), (0, 1))
[2234] ((0, 6), (5, 3), (0, 1))
[2235] ((0, 6), (5, 4), (0, 1))
[2236] ((0, 6), (5, 5), (0, 1))
[2237] ((0, 6), (5, 6), (0, 1))
[2238] ((0, 6), (6, 0), (0, 1))
[2239] ((0, 6), (6, 1), (0, 1))
[2240] ((0, 6), (6, 2), (0, 1))
[2241] ((0, 6), (6, 3), (0, 1))
[2242] ((0, 6), (6, 4), (0, 1))
[2243] ((0, 6), (6, 5), (0, 1))
[2244] ((0, 6), (6, 6), (0, 1))
[2245] ((1, 0), (0, 0), (0, 1))
[2246] (

[3715] ((6, 1), (2, 6), (0, 1))
[3716] ((6, 1), (3, 0), (0, 1))
[3717] ((6, 1), (3, 2), (0, 1))
[3718] ((6, 1), (3, 3), (0, 1))
[3719] ((6, 1), (3, 5), (0, 1))
[3720] ((6, 1), (3, 6), (0, 1))
[3721] ((6, 1), (4, 0), (0, 1))
[3722] ((6, 1), (4, 1), (0, 1))
[3723] ((6, 1), (4, 2), (0, 1))
[3724] ((6, 1), (4, 3), (0, 1))
[3725] ((6, 1), (4, 4), (0, 1))
[3726] ((6, 1), (4, 5), (0, 1))
[3727] ((6, 1), (4, 6), (0, 1))
[3728] ((6, 1), (5, 0), (0, 1))
[3729] ((6, 1), (5, 1), (0, 1))
[3730] ((6, 1), (5, 2), (0, 1))
[3731] ((6, 1), (5, 3), (0, 1))
[3732] ((6, 1), (5, 4), (0, 1))
[3733] ((6, 1), (5, 5), (0, 1))
[3734] ((6, 1), (5, 6), (0, 1))
[3735] ((6, 1), (6, 0), (0, 1))
[3736] ((6, 1), (6, 2), (0, 1))
[3737] ((6, 1), (6, 3), (0, 1))
[3738] ((6, 1), (6, 4), (0, 1))
[3739] ((6, 1), (6, 5), (0, 1))
[3740] ((6, 1), (6, 6), (0, 1))
[3741] ((6, 2), (0, 0), (0, 1))
[3742] ((6, 2), (0, 2), (0, 1))
[3743] ((6, 2), (0, 3), (0, 1))
[3744] ((6, 2), (0, 4), (0, 1))
[3745] ((6, 2), (0, 5), (0, 1))
[3746] (

...

[43703] ((0, 3), (1, 5), (3, 3))
[43704] ((0, 3), (1, 6), (3, 3))
[43705] ((0, 3), (2, 0), (3, 3))
[43706] ((0, 3), (2, 1), (3, 3))
[43707] ((0, 3), (2, 2), (3, 3))
[43708] ((0, 3), (2, 3), (3, 3))
[43709] ((0, 3), (2, 4), (3, 3))
[43710] ((0, 3), (2, 5), (3, 3))
[43711] ((0, 3), (2, 6), (3, 3))
[43712] ((0, 3), (3, 0), (3, 3))
[43713] ((0, 3), (3, 2), (3, 3))
[43714] ((0, 3), (3, 5), (3, 3))
[43715] ((0, 3), (3, 6), (3, 3))
[43716] ((0, 3), (4, 0), (3, 3))
[43717] ((0, 3), (4, 1), (3, 3))
[43718] ((0, 3), (4, 2), (3, 3))
[43719] ((0, 3), (4, 3), (3, 3))
[43720] ((0, 3), (4, 4), (3, 3))
[43721] ((0, 3), (4, 5), (3, 3))
[43722] ((0, 3), (4, 6), (3, 3))
[43723] ((0, 3), (5, 0), (3, 3))
[43724] ((0, 3), (5, 1), (3, 3))
[43725] ((0, 3), (5, 2), (3, 3))
[43726] ((0, 3), (5, 3), (3, 3))
[43727] ((0, 3), (5, 4), (3, 3))
[43728] ((0, 3), (5, 5), (3, 3))
[43729] ((0, 3), (5, 6), (3, 3))
[43730] ((0, 3), (6, 0), (3, 3))
[43731] ((0, 3), (6, 1), (3, 3))
[43732] ((0, 3), (6, 2), (3, 3))
[43733] ((

[45446] ((6, 4), (5, 6), (3, 3))
[45447] ((6, 4), (6, 0), (3, 3))
[45448] ((6, 4), (6, 1), (3, 3))
[45449] ((6, 4), (6, 2), (3, 3))
[45450] ((6, 4), (6, 3), (3, 3))
[45451] ((6, 4), (6, 5), (3, 3))
[45452] ((6, 4), (6, 6), (3, 3))
[45453] ((6, 5), (0, 0), (3, 3))
[45454] ((6, 5), (0, 1), (3, 3))
[45455] ((6, 5), (0, 2), (3, 3))
[45456] ((6, 5), (0, 3), (3, 3))
[45457] ((6, 5), (0, 4), (3, 3))
[45458] ((6, 5), (0, 5), (3, 3))
[45459] ((6, 5), (0, 6), (3, 3))
[45460] ((6, 5), (1, 0), (3, 3))
[45461] ((6, 5), (1, 2), (3, 3))
[45462] ((6, 5), (1, 3), (3, 3))
[45463] ((6, 5), (1, 4), (3, 3))
[45464] ((6, 5), (1, 5), (3, 3))
[45465] ((6, 5), (1, 6), (3, 3))
[45466] ((6, 5), (2, 0), (3, 3))
[45467] ((6, 5), (2, 1), (3, 3))
[45468] ((6, 5), (2, 2), (3, 3))
[45469] ((6, 5), (2, 3), (3, 3))
[45470] ((6, 5), (2, 4), (3, 3))
[45471] ((6, 5), (2, 5), (3, 3))
[45472] ((6, 5), (2, 6), (3, 3))
[45473] ((6, 5), (3, 0), (3, 3))
[45474] ((6, 5), (3, 2), (3, 3))
[45475] ((6, 5), (3, 5), (3, 3))
[45476] ((

[47202] ((5, 6), (5, 2), (3, 5))
[47203] ((5, 6), (5, 3), (3, 5))
[47204] ((5, 6), (5, 4), (3, 5))
[47205] ((5, 6), (5, 5), (3, 5))
[47206] ((5, 6), (6, 0), (3, 5))
[47207] ((5, 6), (6, 1), (3, 5))
[47208] ((5, 6), (6, 2), (3, 5))
[47209] ((5, 6), (6, 3), (3, 5))
[47210] ((5, 6), (6, 4), (3, 5))
[47211] ((5, 6), (6, 5), (3, 5))
[47212] ((5, 6), (6, 6), (3, 5))
[47213] ((6, 0), (0, 0), (3, 5))
[47214] ((6, 0), (0, 1), (3, 5))
[47215] ((6, 0), (0, 2), (3, 5))
[47216] ((6, 0), (0, 3), (3, 5))
[47217] ((6, 0), (0, 4), (3, 5))
[47218] ((6, 0), (0, 5), (3, 5))
[47219] ((6, 0), (0, 6), (3, 5))
[47220] ((6, 0), (1, 0), (3, 5))
[47221] ((6, 0), (1, 2), (3, 5))
[47222] ((6, 0), (1, 3), (3, 5))
[47223] ((6, 0), (1, 4), (3, 5))
[47224] ((6, 0), (1, 5), (3, 5))
[47225] ((6, 0), (1, 6), (3, 5))
[47226] ((6, 0), (2, 0), (3, 5))
[47227] ((6, 0), (2, 1), (3, 5))
[47228] ((6, 0), (2, 2), (3, 5))
[47229] ((6, 0), (2, 3), (3, 5))
[47230] ((6, 0), (2, 4), (3, 5))
[47231] ((6, 0), (2, 5), (3, 5))
[47232] ((

[48926] ((5, 0), (6, 4), (3, 6))
[48927] ((5, 0), (6, 5), (3, 6))
[48928] ((5, 0), (6, 6), (3, 6))
[48929] ((5, 1), (0, 0), (3, 6))
[48930] ((5, 1), (0, 1), (3, 6))
[48931] ((5, 1), (0, 2), (3, 6))
[48932] ((5, 1), (0, 3), (3, 6))
[48933] ((5, 1), (0, 4), (3, 6))
[48934] ((5, 1), (0, 5), (3, 6))
[48935] ((5, 1), (0, 6), (3, 6))
[48936] ((5, 1), (1, 0), (3, 6))
[48937] ((5, 1), (1, 2), (3, 6))
[48938] ((5, 1), (1, 3), (3, 6))
[48939] ((5, 1), (1, 4), (3, 6))
[48940] ((5, 1), (1, 5), (3, 6))
[48941] ((5, 1), (1, 6), (3, 6))
[48942] ((5, 1), (2, 0), (3, 6))
[48943] ((5, 1), (2, 1), (3, 6))
[48944] ((5, 1), (2, 2), (3, 6))
[48945] ((5, 1), (2, 3), (3, 6))
[48946] ((5, 1), (2, 4), (3, 6))
[48947] ((5, 1), (2, 5), (3, 6))
[48948] ((5, 1), (2, 6), (3, 6))
[48949] ((5, 1), (3, 0), (3, 6))
[48950] ((5, 1), (3, 2), (3, 6))
[48951] ((5, 1), (3, 3), (3, 6))
[48952] ((5, 1), (3, 5), (3, 6))
[48953] ((5, 1), (4, 0), (3, 6))
[48954] ((5, 1), (4, 1), (3, 6))
[48955] ((5, 1), (4, 2), (3, 6))
[48956] ((

[50523] ((3, 5), (1, 4), (4, 0))
[50524] ((3, 5), (1, 5), (4, 0))
[50525] ((3, 5), (1, 6), (4, 0))
[50526] ((3, 5), (2, 0), (4, 0))
[50527] ((3, 5), (2, 1), (4, 0))
[50528] ((3, 5), (2, 2), (4, 0))
[50529] ((3, 5), (2, 3), (4, 0))
[50530] ((3, 5), (2, 4), (4, 0))
[50531] ((3, 5), (2, 5), (4, 0))
[50532] ((3, 5), (2, 6), (4, 0))
[50533] ((3, 5), (3, 0), (4, 0))
[50534] ((3, 5), (3, 2), (4, 0))
[50535] ((3, 5), (3, 3), (4, 0))
[50536] ((3, 5), (3, 6), (4, 0))
[50537] ((3, 5), (4, 1), (4, 0))
[50538] ((3, 5), (4, 2), (4, 0))
[50539] ((3, 5), (4, 3), (4, 0))
[50540] ((3, 5), (4, 4), (4, 0))
[50541] ((3, 5), (4, 5), (4, 0))
[50542] ((3, 5), (4, 6), (4, 0))
[50543] ((3, 5), (5, 0), (4, 0))
[50544] ((3, 5), (5, 1), (4, 0))
[50545] ((3, 5), (5, 2), (4, 0))
[50546] ((3, 5), (5, 3), (4, 0))
[50547] ((3, 5), (5, 4), (4, 0))
[50548] ((3, 5), (5, 5), (4, 0))
[50549] ((3, 5), (5, 6), (4, 0))
[50550] ((3, 5), (6, 0), (4, 0))
[50551] ((3, 5), (6, 1), (4, 0))
[50552] ((3, 5), (6, 2), (4, 0))
[50553] ((

In [ ]:
# Here is an example of how to visualise a game replay of 2 random players
game = Board(RandomPlayer(), RandomPlayer(), 7, 7)
winner, move_history, termination = game.play_isolation(time_limit=1000, print_moves=False)

bg = ReplayGame(game, move_history, show_legal_moves=True)
bg.show_board()

# Game AI
---------------------------------

As the game board in Isolation is fully viewable by both players at all times, the game is fully observable, adversarial, and deterministic. This allows the development of an AI-based game playing agent using suitable algorithms and heuristics, such as Minimax and Alphabeta Pruning

## The Minimax Algorithm

Minimax is a decision-making algorithm, **typically used in a turn-based, two player games**. The goal of the algorithm is to find the optimal next move

- **Idea:** Choose move to position with highest minimax value = best achievable payoff against best play
- **Perfect (optimal) play for deterministic, perfect-information games**

### Minimax Algorithm Summary

- In the algorithm, one player is called the maximizer, and the other  player is a minimizer. If we assign an evaluation score to the game board, one player tries to choose a game state with the maximum score, while the other chooses a state with the minimum score


- In other words, the **maximizer works to get the highest score, while the minimizer tries get the lowest score** by trying to counter moves


- It is based on the [zero-sum game](https://en.wikipedia.org/wiki/Zero-sum_game) concept. **In a zero-sum game, the total utility score is divided among the players. An increase in one player's score results into the decrease in another  player's score.** So, the total score is always zero. For one  player to win, the other one has to lose. Examples of such games are chess, poker, checkers, tic-tac-toe


- We traverse the game tree ***depth-first*** until we reach the terminal nodes and assign their parent node a value that is best for the player whose turn it is to move. For example, the game tree of a game of tic-tac-toe looks like:

![](./img/tictactoe-gametree.png)


In [ ]:
## About

In [ ]:
#export
class OpenMoveEvalFn:
    def score(self, game, my_player=None, my_turn=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """
        # Simple crude heuristic which favours the number of possible moves our AI has in comparison
        # to the other player. CustomPlayer is always trying to maximize this value
        # and RandomPlayer is trying to minimimize
        if my_turn:
            my_moves = game.get_active_moves()
            opp_moves = game.get_inactive_moves()

#             num_active_moves_my_player = game.get_player_moves(my_player=my_player)
#             num_active_moves_opponent = game.get_opponent_moves(my_player=my_player)
            
        else:
            my_moves = game.get_inactive_moves()
            opp_moves = game.get_active_moves()
#             num_active_moves_my_player = game.get_opponent_moves(my_player=my_player)
#             num_active_moves_opponent = game.get_player_moves(my_player=my_player)
        
        return  len(my_moves) - len(opp_moves)

######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.correctOpenEvalFn(OpenMoveEvalFn)
################ END OF LOCAL TEST CODE SECTION ######################

In [ ]:
#export
class CustomPlayer:
    # TODO: finish this class!
    """Player that chooses a move using your evaluation function
    and a minimax algorithm with alpha-beta pruning.
    You must finish and test this player to make sure it properly
    uses minimax and alpha-beta to return a good move."""

    def __init__(self, search_depth=3, eval_fn=OpenMoveEvalFn()):
        """Initializes your player.
        
        if you find yourself with a superior eval function, update the default
        value of `eval_fn` to `CustomEvalFn()`
        
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent

        Note:
            1. Do NOT change the name of this 'move' function. We are going to call
            this function directly.
            2. Call alphabeta instead of minimax once implemented.
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: ((int,int),(int,int),(int,int)): Your best move
        """
        best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
        print(f"AI Player: Moving {best_move} with value {utility}")
        print("---------------------------")
        return best_move

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        return self.eval_fn.score(game, self)



###################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE CLASS! ################
###### IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ###########
###################################################################

In [ ]:
# dir(Board)

t1_state = [
     [' ', ' ', ' ', ' ', ' ', ' ', ' '],
     [' ', '11', ' ', ' ', ' ', ' ', ' '],
     [' ', ' ', ' ', ' ', '22', ' ', ' '],
     [' ', ' ', '12', ' ', ' ', ' ', ' '],
     [' ', ' ', ' ', ' ', '23', ' ', ' '],
     [' ', '21', ' ', ' ', ' ', '13', ' '],
     [' ', ' ', ' ', ' ', ' ', ' ', ' ']
]

r1 = RandomPlayer(name="R1")
r2 = RandomPlayer(name="R2")
test_game = Board(r1, r2 , 7, 7)
test_game.set_state(t1_state, p1_turn=True)
print(test_game.get_active_player().get_name())
test_game.get_state()

In [ ]:
# Algorithm for finding the best move
def minimax(player, game, time_left, depth, my_turn=True):
    """Implementation of the minimax algorithm.
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you
            need from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer()).
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val

    # Note:
        depth --> tells us when to stop searching the tree
    # Steps in the algorithm:
    1. Terminal state check. If at the root node, return the utility of the node/state
    2. Get all possible moves for the current player
    3. Determine if a winner is found for any of the moves from step 2
        a. If the winner is MAX, return the move, utility == +inf
        b. if the winner is MIN, return the move, utility == -inf

    4. If no winner is found, continue searching the game tree by recursively calling MINIMAX to get
    the best forecast value
    5. Set best_move, best_value to be highest utility moves based on forecasted value

    """
#     player.count += 1

    # Determine whose turn it is
    if my_turn:
        ai_player = game.get_active_player()
        cpu_player = game.get_inactive_player()

    else:
        ai_player = game.get_inactive_player()
        cpu_player = game.get_active_player()

    ai_moves = game.get_player_moves(my_player=ai_player)
    cpu_moves = game.get_opponent_moves(my_player=ai_player)

    # Handle time running out
    # TODO
    # print(f"Time left {time_left()}")
    #     if time_left() < 10:
    #         return None, None

    ####################################################################################################
    # Search the game tree
    ####################################################################################################

    # If depth is 0, we know we've arrived at the lowest depth desired. Return number of available moves
    # for the AI - number of moves available for the opponent
    if depth == 0:
        best_value = ai_player.utility(game, my_turn)
        return None, best_value

    if my_turn:
        # Initialize values
        max_value = float("-inf")
        best_move = None

        #         # Get possible moves of the CustomPlayer
        #         ai_moves = game.get_player_moves(my_player=current_player)

        for move in ai_moves:
            #             print(f"Possible Move: {move}")

            # Check all possible moves to see if a winner can be found
            new_board_state, is_over, winner = game.forecast_move(move)

            # Check to see if the game is ended while it is the AI's turn and after the next move
            next_moves_possible = new_board_state.get_player_moves(my_player=ai_player)

            if is_over and len(next_moves_possible) == 0:
                #                 print(f"AI Player: Game is over with move {move}")
                return move, float("inf")

            else:
                #                 print(f"Searching game tree for move {move}")
                # Recursively search through the game tree
                forecasted_move, forecasted_value = minimax(
                    ai_player, new_board_state, time_left, depth=depth - 1, my_turn=False
                )

                if forecasted_value > max_value:
                    #                     print(f"Swapping Max Value {max_value} with forecasted value {forecasted_value} with move {forecasted_move}")
                    max_value = forecasted_value
                    best_move = move
                    # print(f"Found new best move: {best_move} with value {max_value}")

        #         print(f"Returning Best Move {best_move} with value {max_value}")
        #         print()
        return best_move, max_value

    else:  # Opponents turn

        # Initialize values
        min_value = float("inf")
        best_move = None

        #         # Get possible moves of the CustomPlayer
        #         cpu_moves = game.get_opponent_moves(my_player=current_player)

        for move in cpu_moves:
            #             print(f"Possible Move: {move}")

            # Check all possible moves to see if a winner can be found
            new_board_state, is_over, winner = game.forecast_move(move)

            # Check to see if the game is ended while it is the opponents turn and after the next move
            next_moves_possible = new_board_state.get_opponent_moves(my_player=ai_player)

            if is_over and len(next_moves_possible) == 0:
                #                 print(f"Opponent Player: Game is over with move {move}")
                return move, float("-inf")
            else:
                #                 print(f"Searching game tree for move {move}")
                # Recursively search through the game tree
                forecasted_move, forecasted_value = minimax(
                    ai_player, new_board_state, time_left, depth=depth - 1, my_turn=True
                )

                if forecasted_value < min_value:
                    #                     print(f"Swapping Max Value {min_value} with forecasted value {forecasted_value} with move {forecasted_move}")
                    min_value = forecasted_value
                    best_move = move
                    print(f"Finding Opponents best move: {best_move} with value {min_value}")

        #         print(f"Returning Best Move {best_move} with value {min_value}")
        #         print()
        return best_move, min_value
    
        


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.beatRandom(CustomPlayer)
tests.algorithmTest(CustomPlayer, minimax, "Minimax")
################ END OF LOCAL TEST CODE SECTION ######################

### Run Time

Minimax Search Depth == 1: Run Time ~ 59 Seconds
Minimax Search Depth == 2: Run Time ~1245  Seconds

Number of Moves Computed:
* Search Depth == 0: 46 Possible Moves (49 Possible moves from start - 3 by first CPU move)
* Search Depth == 1: 45! * 44! * 43!

In [24]:
from interactive_board_viz import PlayInteractiveGame
from isolation import Board
# from board_viz import InteractiveGame
from custom_player import CustomPlayer

##### NOTE: Need to click the squares order in which they move

# Remove the first argument ('RandomPlayer()') entirely to play against another HumanPlayer
ai_player = CustomPlayer(search_depth=1)
ig = PlayInteractiveGame(player1 = RandomPlayer(), opponent = ai_player, show_legal_moves=True)
# ig = InteractiveGame(opponent = RandomPlayer(), show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

Output(layout=Layout(border='1px solid black'))

Button(description='Start', layout=Layout(height='50', width='100'), style=ButtonStyle())

Output()

In [ ]:
ai_player.count

In [ ]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

# TODO

- Try iterative deepening approach
- Implement Alpha-Beta Pruning
- Try using transposition tables